In [ ]:
#Import packages
import os
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
import importlib
import openpyxl
from scipy.stats import mannwhitneyu

In [ ]:
from functions import *

In [ ]:
# Set the directory for the data
results_dir = 'C:/Users/ifiri/Documents/PROYECTOS/2024_claudia_alzheimer/results'
data_dir = 'C:/Users/ifiri/Documents/PROYECTOS/2024_claudia_alzheimer/precuneus_nets'

In [ ]:
# Separate the data into 2 groups: (1) aC and (2) nC
xl = pd.ExcelFile('sujetos_time_series_codes.xlsx')
df = xl.parse('datos_demograficos_sanos_schaef')

In [ ]:
# Quiero sacar los indices de los sujetos aC y nC
aC_index = df[df['CONDICIÓN']=='aC']
nC_index = df[df['CONDICIÓN']=='nC']

In [ ]:
# Directory containing the data
os.chdir(data_dir)

# Load the time series data
ts_data, ts_data_struc, struc_names = process_data(data_dir)

# Select the data for the aC group using the indices
ts_aC_data = ts_data[aC_index.index]

# Select the data for the nC group using the indices
ts_nC_data = ts_data[nC_index.index]

In [ ]:
# SELECT THE GROUP TO ANALYZE
ts_data = ts_nC_data
group = 'nC'

In [ ]:
# Window the data and compute the correlation matrices
structure_names = struc_names.to_list()
delta = 8 #number of time points on each window. LB^(-1) = delta * TR (2.2s)
corr_matrices = split_into_windows_and_compute_correlation(ts_data, delta)
print(corr_matrices.shape)  # Debería ser (62, 62, 57, 33)

Example plots and data exploration analysis

In [ ]:
# Access a specific value
subject_id = 'Subject001'
timepoint = 1
structure = 'atlas.Precuneous'

specific_value = ts_data_struc.sel(subject=subject_id, timepoint=timepoint, struc=structure).item()
print(f"The value for {subject_id}, timepoint {timepoint}, structure '{structure}' is {specific_value}.")

In [ ]:
# plot a heatmap (sns) of a correlation matrix (e.g. the first one) from corr_df and add the x and y labels of the structure_names

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrices[:, :, 0, 0], xticklabels=structure_names, yticklabels=structure_names, cmap='viridis')
plt.xlabel('Structures')
plt.ylabel('Structures')
plt.title('Correlation matrix for the first window of the first subject')
plt.show()


In [ ]:
# Make a panel of figures with the correlation matrices for the first subject and all windows (33 windows) a grid of 6x6 figures

fig, axs = plt.subplots(6, 6, figsize=(25, 25))
for i, ax in enumerate(axs.flatten()):
    sns.heatmap(corr_matrices[:, :, 0, i], cmap='viridis', ax=ax)
    ax.set_title(f'Window {i+1}')



In [ ]:
# Access a specific structure
structure = 'atlas.Precuneous'

# Select the data for the specified structure
specific_structure_data = ts_data_struc.sel(struc=structure)
print(f"Data for structure '{structure}':\n", specific_structure_data)

INTER-PRECUNEUS ANALYSIS: 6 ANALYSIS IN TOTAL
(1) precuneus (total) with the rest
(2-6) 7Am, 7Pm, 7m, PVC and POS2 with the rest 

In [ ]:
# Fix the structure
structure = 'POS2'

In [ ]:
# We will fix one structure and perfom the analysis
structure_index = structure_names.index(structure)

# Seleccionar todas las estructuras excepto la fijada
remaining_indices = [i for i in range(len(structure_names)) if i != structure_index]

# Seleccionar los datos correspondientes a la fila de la estructura fijada y las columnas restantes
corr_matrices_structure_rows = corr_matrices[remaining_indices,:, :, :]
corr_matrices_structure_col = corr_matrices_structure_rows[:,structure_index, :, :]
print(corr_matrices_structure_col[:,2,20]) # Debería ser un vector de 62 elementos. El elemento 6 (precuneus) debería tener valor de corr=1. La correlacion de una estructura consigo misma (6,6) es 1.

# I want to compute the euclidean distance between all pairs of 33 windows so that i obtain a 33x33 matrix for each subject. 
# I will have 57 subjects, so the final matrix will be 33x33x57.

norm_distances = compute_norm_distances(corr_matrices_structure_col)
print(norm_distances.shape)  # Debería ser (33, 33, 57)

# Flatten each subject distance matrix to obtain a vector per subject

flattened_distances = flatten_distance_matrices(norm_distances)
print(flattened_distances.shape)  # Debería ser (57, 1089)

# Mean distance per subject
mean_distances = np.mean(flattened_distances, axis=1)

# Save the mean distances to a CSV file in the main folder and include fixed structure name in the filename
filename = f'{group}_inter_analysis_mean_distances_{structure}.csv'
np.savetxt(os.path.join(results_dir,filename), mean_distances, delimiter=',')
print(f"Mean distances saved to '{filename}'.")

INTRA-PRECUNEUS ANALYSIS: 1 IN TOTAL.
7Am, 7Pm, 7m, PCV and POS2 strucs between them

In [ ]:
# Fix the 5 structures and find the index of each one
strucs =  '7Am', '7Pm', '7m', 'PVC', 'POS2'

In [ ]:
# We will fix one structure and perfom the analysis
structure_indices = [structure_names.index(s) for s in strucs]

# Select the data for the specified structure. The final matrix should be 5x5x57x33
corr_matrices_structure_row = corr_matrices[structure_indices, :, :, :]
corr_matrices_structure_col = corr_matrices_structure_row[:, structure_indices, :, :]
print(corr_matrices_structure_col.shape)  # Debe ser: (5,5,57,33)

# Flatten the intra-structure matrices to compute the norm distance between vectors and no between matrices
intra_size = corr_matrices_structure_col.shape[0]
subjects = corr_matrices_structure_col.shape[2]
windows = corr_matrices_structure_col.shape[3]
corr_matrices_structure_flat = corr_matrices_structure_col.reshape(intra_size*intra_size, subjects, windows)

# I want to compute the euclidean distance between all pairs of 33 windows so that i obtain a 33x33 matrix for each subject. 
# I will have 57 subjects, so the final matrix will be 33x33x57.

norm_distances = compute_norm_distances(corr_matrices_structure_flat)
print(norm_distances.shape)  # Debería ser (33, 33, 57)

# Flatten each subject distance matrix to obtain a vector per subject

flattened_distances = flatten_distance_matrices(norm_distances)
print(flattened_distances.shape)  # Debería ser (57, 1089)

# Mean distance per subject
mean_distances = np.mean(flattened_distances, axis=1)

# Save the mean distances to a CSV file in the main folder and include fixed structure name in the filename
filename = f'{group}_intra_analysis_mean_distances_5precuneus.csv'
np.savetxt(os.path.join(results_dir,filename), mean_distances, delimiter=',')
print(f"Mean distances saved to '{filename}'.")

Non-parametric t-test between groups (aC vs. nC): 7 comparisons. (1) intra-5precuneus (6) inter-precuneus, 7Am, 7Pm, 7m, PVC, POS2

In [ ]:
# Load csv file as vector
structure = '5precuneus'

file_aC = f'aC_intra_analysis_mean_distances_{structure}.csv'
file_nC = f'nC_intra_analysis_mean_distances_5precuneus.csv'
mean_distances_aC = np.loadtxt(os.path.join(results_dir,file_aC), delimiter=',')
mean_distances_nC = np.loadtxt(os.path.join(results_dir,file_nC), delimiter=',')

In [ ]:
# Compute a non parametric test to compare the mean distances between the two groups

# Perform the Mann-Whitney U test
# Assumptions:
## (1) Independence: The samples must be independent of each other.
## (2) Ordinal or Continuous Data: The data should be at least ordinal (ranked) or continuous.

from scipy.stats import mannwhitneyu
stat, p_value = mannwhitneyu(mean_distances_aC, mean_distances_nC, alternative='two-sided')

print('Statistics:', stat)
print('p-value:', p_value)


In [ ]:
# Perform the independent t-test
# Assumptions:    
## (1) Normality: The data in each group should be approximately normally distributed.
## (2) Independence: The samples must be independent of each other.
## (3) Homogeneity of Variance: The variances of the two groups should be equal.

from scipy.stats import ttest_ind
stat, p_value = ttest_ind(mean_distances_aC, mean_distances_nC,  equal_var=False)
print('Statistics:', stat)
print('p-value:', p_value)

RESULTS COMPARISON

In [ ]:
import pandas as pd
from scipy.stats import mannwhitneyu
import os

# Directory containing the CSV files
results_dir = "C:/Users/ifiri/Documents/PROYECTOS/2024_claudia_alzheimer/results/"

# Initialize lists to store filenames
aC_files = []
nC_files = []

# Separate filenames into aC and nC lists
for filename in os.listdir(results_dir):
    if filename.startswith("aC"):
        aC_files.append(filename)
    elif filename.startswith("nC"):
        nC_files.append(filename)

# Extract structure names without prefixes and suffixes for matching
aC_structures = [filename.split('_')[5].split('.')[0] for filename in aC_files]
nC_structures = [filename.split('_')[5].split('.')[0] for filename in nC_files]

# Initialize lists to store results
results = []

# Perform Mann-Whitney U test for 5precuneus (intra-analysis)
ac_file = "aC_intra_analysis_mean_distances_5precuneus.csv"
nc_file = "nC_intra_analysis_mean_distances_5precuneus.csv"
ac_data = pd.read_csv(os.path.join(results_dir, ac_file))
nc_data = pd.read_csv(os.path.join(results_dir, nc_file))

ac_values = ac_data.iloc[:, 0]
nc_values = nc_data.iloc[:, 0]

stat, p_value = mannwhitneyu(ac_values, nc_values, alternative='two-sided')

results.append({'Structure': '5precuneus', 'U-statistic': stat, 'p-value': p_value, 'Comparison': 'Intra'})

# Perform Mann-Whitney U test for each matching structure (inter-analysis)
for structure in set(aC_structures) & set(nC_structures):
    if structure == '5precuneus':
        continue
    
    aC_file = next(f for f in aC_files if structure in f)
    nC_file = next(f for f in nC_files if structure in f)
    
    # Load data for the current structure
    aC_data = pd.read_csv(os.path.join(results_dir, aC_file))
    nC_data = pd.read_csv(os.path.join(results_dir, nC_file))

    # Assuming the data is in a single column, if there are multiple columns, specify the correct one
    aC_values = aC_data.iloc[:, 0]
    nC_values = nC_data.iloc[:, 0]

    # Perform Mann-Whitney U test
    stat, p_value = mannwhitneyu(aC_values, nC_values, alternative='two-sided')

    # Append results
    results.append({'Structure': structure, 'U-statistic': stat, 'p-value': p_value, 'Comparison': 'Inter'})

# Create a DataFrame to display results
results_df = pd.DataFrame(results)

# Apply Bonferroni correction
num_comparisons = len(results_df)
bonferroni_alpha = 0.05 / num_comparisons
results_df['corrected p-value'] = results_df['p-value'] * num_comparisons

# Ensure that corrected p-values do not exceed 1
results_df['corrected p-value'] = results_df['corrected p-value'].apply(lambda p: min(p, 1.0))

# Determine if the corrected p-value is significant
results_df['significant'] = results_df['corrected p-value'] < bonferroni_alpha

print('Bonferroni alpha: 0.05/num.comparisons =', bonferroni_alpha)
print(results_df)
